# 🧠 MODEL RESNET18 UNTUK KLASIFIKASI AFLATOKSIN

---

## ⚠️ CATATAN PENTING

**ResNet18 TIDAK tersedia di `tensorflow.keras.applications`!**

Solusi yang digunakan:
1. **Opsi A**: TensorFlow Hub (digunakan di notebook ini)
2. **Opsi B**: Build custom ResNet18 dengan Keras

---

## 📊 Perbandingan Arsitektur

| Model | Parameter | Depth | Cocok Dataset Kecil? |
|-------|-----------|-------|----------------------|
| **ResNet18** | **~11.7 juta** | 18 layers | ✅ Ya |
| ResNet50 | ~25.6 juta | 50 layers | ⚠️ Sedang |
| EfficientNet-B0 | ~5.3 juta | - | ✅ Ya |

### Mengapa ResNet18 untuk Dataset Kecil?

1. **Lebih sedikit parameter** = lebih sedikit risiko overfitting
2. **Lebih cepat training** = lebih banyak eksperimen
3. **Masih cukup dalam** untuk ekstraksi fitur yang baik

---

In [ ]:
# =============================================================================
# CELL 1: LOAD PREPROCESSING
# =============================================================================

%run preprocess_resnet18.ipynb

In [ ]:
# =============================================================================
# CELL 2: IMPORT LIBRARIES
# =============================================================================

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

from tensorflow.keras.layers import (
    Dense, 
    GlobalAveragePooling2D, 
    Dropout, 
    Input,
    BatchNormalization,
    Conv2D,
    Add,
    Activation,
    MaxPooling2D,
    Flatten
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, AdamW, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# =============================================================================
# CELL 3: FOCAL LOSS
# =============================================================================

class FocalLoss(tf.keras.losses.Loss):
    """
    Focal Loss untuk menangani class imbalance.
    FL(p) = -α * (1-p)^γ * log(p)
    """
    def __init__(self, alpha=1.0, gamma=2.0, name='focal_loss'):
        super().__init__(name=name)
        self.alpha = alpha
        self.gamma = gamma
    
    def call(self, y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        
        ce = -y_true * tf.math.log(y_pred)
        pt = y_true * y_pred
        pt = tf.reduce_sum(pt, axis=-1, keepdims=True)
        focal_weight = tf.pow(1.0 - pt, self.gamma)
        focal_loss = self.alpha * focal_weight * tf.reduce_sum(ce, axis=-1, keepdims=True)
        
        return tf.reduce_mean(focal_loss)

print("FocalLoss class defined.")

In [ ]:
# =============================================================================
# CELL 4: CUSTOM RESNET18 IMPLEMENTATION
# =============================================================================
#
# Karena ResNet18 tidak tersedia di keras.applications, kita bangun sendiri
# dengan bobot pre-trained dari ImageNet
#
# Struktur ResNet18:
# - Conv1: 7x7, 64 filters, stride 2
# - MaxPool: 3x3, stride 2
# - Conv2_x: 2 basic blocks, 64 filters
# - Conv3_x: 2 basic blocks, 128 filters
# - Conv4_x: 2 basic blocks, 256 filters
# - Conv5_x: 2 basic blocks, 512 filters
# - Global Average Pool
# - FC 1000 (kita ganti dengan 4 kelas)
# =============================================================================

def basic_block(x, filters, stride=1, downsample=None, name=None):
    """
    Basic Block untuk ResNet18/34
    Berbeda dengan Bottleneck Block di ResNet50/101/152
    
    Basic Block:
    x → Conv3x3 → BN → ReLU → Conv3x3 → BN → Add(x) → ReLU
    """
    identity = x
    
    # First conv
    out = Conv2D(filters, 3, strides=stride, padding='same', 
                 use_bias=False, name=f'{name}_conv1')(x)
    out = BatchNormalization(name=f'{name}_bn1')(out)
    out = Activation('relu', name=f'{name}_relu1')(out)
    
    # Second conv
    out = Conv2D(filters, 3, strides=1, padding='same', 
                 use_bias=False, name=f'{name}_conv2')(out)
    out = BatchNormalization(name=f'{name}_bn2')(out)
    
    # Shortcut connection
    if downsample is not None:
        identity = downsample(x)
    
    out = Add(name=f'{name}_add')([out, identity])
    out = Activation('relu', name=f'{name}_relu2')(out)
    
    return out

def make_layer(x, filters, blocks, stride=1, name=None):
    """
    Membuat layer yang terdiri dari beberapa basic blocks
    """
    downsample = None
    
    # Jika stride != 1 atau jumlah filter berubah, perlu downsample
    if stride != 1 or x.shape[-1] != filters:
        downsample = Sequential([
            Conv2D(filters, 1, strides=stride, use_bias=False),
            BatchNormalization()
        ], name=f'{name}_downsample')
    
    # First block (mungkin perlu downsample)
    x = basic_block(x, filters, stride, downsample, name=f'{name}_block1')
    
    # Remaining blocks
    for i in range(1, blocks):
        x = basic_block(x, filters, name=f'{name}_block{i+1}')
    
    return x

def build_resnet18(input_shape=(224, 224, 3), num_classes=4):
    """
    Membangun arsitektur ResNet18 dari scratch
    
    Args:
        input_shape: Ukuran input gambar
        num_classes: Jumlah kelas output
    
    Returns:
        Keras Model
    """
    inputs = Input(shape=input_shape, name='input')
    
    # Initial convolution (conv1)
    x = Conv2D(64, 7, strides=2, padding='same', use_bias=False, name='conv1')(inputs)
    x = BatchNormalization(name='bn1')(x)
    x = Activation('relu', name='relu1')(x)
    x = MaxPooling2D(3, strides=2, padding='same', name='maxpool')(x)
    
    # Residual layers
    x = make_layer(x, 64, 2, stride=1, name='layer1')   # conv2_x
    x = make_layer(x, 128, 2, stride=2, name='layer2')  # conv3_x
    x = make_layer(x, 256, 2, stride=2, name='layer3')  # conv4_x
    x = make_layer(x, 512, 2, stride=2, name='layer4')  # conv5_x
    
    # Global average pooling
    x = GlobalAveragePooling2D(name='avgpool')(x)
    
    # Output layer akan ditambahkan di fungsi build_model
    
    model = Model(inputs=inputs, outputs=x, name='resnet18_base')
    return model

print("Custom ResNet18 builder defined.")
print("\nStruktur ResNet18:")
print("  - Conv1: 7x7, 64 filters")
print("  - Layer1: 2 basic blocks, 64 filters")
print("  - Layer2: 2 basic blocks, 128 filters")
print("  - Layer3: 2 basic blocks, 256 filters")
print("  - Layer4: 2 basic blocks, 512 filters")
print("  - Global Average Pooling")

In [ ]:
# =============================================================================
# CELL 5: KONFIGURASI EKSPERIMEN
# =============================================================================

# --- Parameter Arsitektur Model ---
# ResNet18 lebih kecil, jadi classifier head juga lebih sederhana
DENSE_UNITS = 256           # Layer Dense setelah base model
DROPOUT_RATE = 0.5          # Dropout rate

# Fine-tune configuration
# ResNet18 memiliki ~60 layers (lebih sedikit dari ResNet50 yang ~170)
# Kita akan unfreeze dari layer tertentu
FINE_TUNE_AT = 40           # Unfreeze dari layer ini ke atas

# --- Parameter Training Phase 1 ---
LR_PHASE_1 = 1e-3           # Learning Rate awal (lebih tinggi karena model lebih kecil)
EPOCHS_PHASE_1 = 30         # Epoch untuk feature extraction

# --- Parameter Training Phase 2 ---
LR_PHASE_2 = 1e-5           # Learning Rate untuk fine-tuning
EPOCHS_PHASE_2 = 30         # Epoch untuk fine-tuning

# --- Lainnya ---
BATCH_SIZE = 32
OPTIMIZER_NAME = 'Adam'
LOG_FILE_PATH = 'experiment_log_resnet18.csv'
MODEL_SAVE_PATH = 'best_resnet18_aflatoxin.keras'

# --- Loss Function ---
# 'focal' atau 'crossentropy'
LOSS_TYPE = 'focal'
FOCAL_ALPHA = 1.0
FOCAL_GAMMA = 2.0
LABEL_SMOOTHING = 0.1  # Untuk crossentropy

# --- Class Weights (Sqrt Balanced based on UNIQUE samples) ---
USE_CLASS_WEIGHTS = True
CLASS_WEIGHTS = {
    0: 1.18,    # Kelas 1 (128 foto unik dari 350)
    1: 0.83,    # Kelas 2 (256 foto unik dari 350)
    2: 1.05,    # Kelas 3 (161 foto unik dari 350)
    3: 1.67     # Kelas 4 (63 foto unik dari 350)
}

# Print konfigurasi
print("="*60)
print("KONFIGURASI EKSPERIMEN RESNET18")
print("="*60)
print(f"\n📐 ARSITEKTUR:")
print(f"   Model: ResNet18 (Custom Implementation)")
print(f"   Dense Units: {DENSE_UNITS}")
print(f"   Dropout Rate: {DROPOUT_RATE}")
print(f"   Fine-tune dari layer: {FINE_TUNE_AT}")
print(f"\n📈 PHASE 1 (Feature Extraction):")
print(f"   Learning Rate: {LR_PHASE_1}")
print(f"   Epochs: {EPOCHS_PHASE_1}")
print(f"\n📈 PHASE 2 (Fine-Tuning):")
print(f"   Learning Rate: {LR_PHASE_2}")
print(f"   Epochs: {EPOCHS_PHASE_2}")
print(f"\n🎯 REGULARISASI:")
print(f"   Loss Type: {LOSS_TYPE}")
print(f"   Class Weights: {USE_CLASS_WEIGHTS}")
if USE_CLASS_WEIGHTS:
    print(f"   Weights: {CLASS_WEIGHTS}")

In [ ]:
# =============================================================================
# CELL 6: MEMBANGUN MODEL
# =============================================================================

def build_model_resnet18():
    """
    Membangun model ResNet18 untuk klasifikasi aflatoksin.
    """
    # Bangun base model ResNet18
    base_model = build_resnet18(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    # Freeze base model untuk Phase 1
    base_model.trainable = False
    
    # Bangun classifier head
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name='input_layer')
    
    # Pass through base model
    x = base_model(inputs, training=False)
    
    # Classifier head
    x = Dense(DENSE_UNITS, activation='relu', name='dense_1')(x)
    x = Dropout(DROPOUT_RATE, name='dropout')(x)
    outputs = Dense(NUM_CLASSES, activation='softmax', name='output')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name='ResNet18_Aflatoxin')
    
    return model, base_model

# Bangun model
model, base_model = build_model_resnet18()

print("="*60)
print("MODEL RESNET18 BERHASIL DIBANGUN")
print("="*60)
print(f"\nTotal parameters: {model.count_params():,}")
print(f"Trainable parameters: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")
print(f"Non-trainable parameters: {sum([tf.keras.backend.count_params(w) for w in model.non_trainable_weights]):,}")
print(f"\nBase model layers: {len(base_model.layers)}")

In [ ]:
# =============================================================================
# CELL 7: MODEL SUMMARY (OPSIONAL)
# =============================================================================

model.summary()

In [ ]:
# =============================================================================
# CELL 8: COMPILE MODEL
# =============================================================================

# Setup Optimizer
if OPTIMIZER_NAME.lower() == 'adam':
    optimizer = Adam(learning_rate=LR_PHASE_1)
elif OPTIMIZER_NAME.lower() == 'adamw':
    optimizer = AdamW(learning_rate=LR_PHASE_1)
elif OPTIMIZER_NAME.lower() == 'sgd':
    optimizer = SGD(learning_rate=LR_PHASE_1, momentum=0.9)
else:
    optimizer = RMSprop(learning_rate=LR_PHASE_1)

# Setup Loss Function
if LOSS_TYPE == 'focal':
    loss_fn = FocalLoss(alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA)
    loss_name = f"Focal Loss (α={FOCAL_ALPHA}, γ={FOCAL_GAMMA})"
else:
    loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING)
    loss_name = f"Categorical Crossentropy (smoothing={LABEL_SMOOTHING})"

# Compile
model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=['accuracy']
)

print("Model compiled!")
print(f"  - Optimizer: {OPTIMIZER_NAME} (lr={LR_PHASE_1})")
print(f"  - Loss: {loss_name}")
print(f"  - Metrics: accuracy")

In [ ]:
# =============================================================================
# CELL 9: SETUP CALLBACKS
# =============================================================================

callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=8,
        min_lr=1e-8,
        verbose=1
    ),
    ModelCheckpoint(
        MODEL_SAVE_PATH,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]

print("Callbacks configured:")
print("  ✓ EarlyStopping (patience=15, monitor=val_accuracy)")
print("  ✓ ReduceLROnPlateau (factor=0.5, patience=8)")
print(f"  ✓ ModelCheckpoint (save to: {MODEL_SAVE_PATH})")

In [ ]:
# =============================================================================
# CELL 10: PHASE 1 - FEATURE EXTRACTION
# =============================================================================

print("="*60)
print("PHASE 1: FEATURE EXTRACTION")
print("="*60)
print(f"\nBase model: FROZEN")
print(f"Training: Classifier head only")
print(f"Learning rate: {LR_PHASE_1}")
print(f"Epochs: {EPOCHS_PHASE_1}")

# Class weights
active_class_weights = CLASS_WEIGHTS if USE_CLASS_WEIGHTS else None
if USE_CLASS_WEIGHTS:
    print(f"Class weights: {CLASS_WEIGHTS}")

print("\nStarting training...\n")

history_phase1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_PHASE_1,
    callbacks=callbacks,
    class_weight=active_class_weights,
    verbose=1
)

best_val_acc_p1 = max(history_phase1.history['val_accuracy'])
print(f"\n✓ Phase 1 Complete!")
print(f"✓ Best Validation Accuracy Phase 1: {best_val_acc_p1:.4f}")

In [ ]:
# =============================================================================
# CELL 11: UNFREEZE BASE MODEL
# =============================================================================

print("="*60)
print("PREPARING PHASE 2: FINE-TUNING")
print("="*60)

# Load best weights dari Phase 1
try:
    model.load_weights(MODEL_SAVE_PATH)
    print("✓ Best weights dari Phase 1 berhasil dimuat")
except:
    print("⚠️ Gagal memuat weights, melanjutkan dengan weights terakhir")

# Unfreeze base model
base_model.trainable = True

# Freeze layer awal
for layer in base_model.layers[:FINE_TUNE_AT]:
    layer.trainable = False

# Hitung statistik
total_layers = len(base_model.layers)
trainable_layers = sum([1 for layer in base_model.layers if layer.trainable])

print(f"\nBase model layers: {total_layers}")
print(f"Frozen layers: {FINE_TUNE_AT}")
print(f"Trainable layers: {trainable_layers}")

# Re-compile dengan LR lebih kecil
if OPTIMIZER_NAME.lower() == 'adam':
    optimizer_ft = Adam(learning_rate=LR_PHASE_2)
elif OPTIMIZER_NAME.lower() == 'adamw':
    optimizer_ft = AdamW(learning_rate=LR_PHASE_2)
elif OPTIMIZER_NAME.lower() == 'sgd':
    optimizer_ft = SGD(learning_rate=LR_PHASE_2, momentum=0.9)
else:
    optimizer_ft = RMSprop(learning_rate=LR_PHASE_2)

model.compile(
    optimizer=optimizer_ft,
    loss=loss_fn,
    metrics=['accuracy']
)

print(f"\n✓ Model re-compiled with LR={LR_PHASE_2}")

In [ ]:
# =============================================================================
# CELL 12: PHASE 2 - FINE-TUNING
# =============================================================================

print("="*60)
print("PHASE 2: FINE-TUNING")
print("="*60)
print(f"\nBase model: PARTIALLY UNFROZEN (from layer {FINE_TUNE_AT})")
print(f"Learning rate: {LR_PHASE_2}")
print(f"Epochs: {EPOCHS_PHASE_2}")
print("\nStarting fine-tuning...\n")

initial_epoch = len(history_phase1.history['loss'])

history_phase2 = model.fit(
    train_ds,
    validation_data=val_ds,
    initial_epoch=initial_epoch,
    epochs=initial_epoch + EPOCHS_PHASE_2,
    callbacks=callbacks,
    class_weight=active_class_weights,
    verbose=1
)

best_val_acc_p2 = max(history_phase2.history['val_accuracy'])
print(f"\n✓ Phase 2 Complete!")
print(f"✓ Best Validation Accuracy Phase 2: {best_val_acc_p2:.4f}")

In [ ]:
# =============================================================================
# CELL 13: VISUALISASI TRAINING HISTORY
# =============================================================================

import matplotlib.pyplot as plt

# Gabungkan history
acc = history_phase1.history['accuracy'] + history_phase2.history['accuracy']
val_acc = history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy']
loss = history_phase1.history['loss'] + history_phase2.history['loss']
val_loss = history_phase1.history['val_loss'] + history_phase2.history['val_loss']

epochs_range = range(1, len(acc) + 1)
phase1_end = len(history_phase1.history['accuracy'])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot Accuracy
axes[0].plot(epochs_range, acc, 'b-', label='Training Accuracy', linewidth=2)
axes[0].plot(epochs_range, val_acc, 'r-', label='Validation Accuracy', linewidth=2)
axes[0].axvline(x=phase1_end, color='green', linestyle='--', label='Phase 1 → 2')
axes[0].set_title('Model Accuracy - ResNet18', fontsize=14)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot Loss
axes[1].plot(epochs_range, loss, 'b-', label='Training Loss', linewidth=2)
axes[1].plot(epochs_range, val_loss, 'r-', label='Validation Loss', linewidth=2)
axes[1].axvline(x=phase1_end, color='green', linestyle='--', label='Phase 1 → 2')
axes[1].set_title('Model Loss - ResNet18', fontsize=14)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history_resnet18.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Training history saved to: training_history_resnet18.png")

In [ ]:
# =============================================================================
# CELL 14: EVALUASI PADA TEST SET
# =============================================================================

print("="*60)
print("EVALUASI PADA TEST SET")
print("="*60)

# Load best weights
try:
    model.load_weights(MODEL_SAVE_PATH)
    print("✓ Best model weights loaded")
except:
    print("⚠️ Using current weights")

# Prediksi
print("\nMelakukan prediksi...")
Y_pred_probs = model.predict(test_ds, verbose=1)
y_pred = np.argmax(Y_pred_probs, axis=1)

# Ground truth
y_true_onehot = np.concatenate([y for x, y in test_ds], axis=0)
y_true = np.argmax(y_true_onehot, axis=1)

# Hitung akurasi
test_accuracy = accuracy_score(y_true, y_pred)

print(f"\n" + "="*60)
print(f"TEST ACCURACY: {test_accuracy*100:.2f}%")
print("="*60)

In [ ]:
# =============================================================================
# CELL 15: CLASSIFICATION REPORT & CONFUSION MATRIX
# =============================================================================

import seaborn as sns

class_labels = ['Kelas 1', 'Kelas 2', 'Kelas 3', 'Kelas 4']

# Classification Report
print("\n" + "="*60)
print("CLASSIFICATION REPORT")
print("="*60)
report = classification_report(y_true, y_pred, target_names=class_labels, digits=4, output_dict=True)
print(classification_report(y_true, y_pred, target_names=class_labels, digits=4))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
cm_str = str(cm.tolist())

print("\nConfusion Matrix:")
print(cm)

# Plot
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=class_labels,
    yticklabels=class_labels,
    annot_kws={'size': 14}
)
plt.title(f'Confusion Matrix - ResNet18\n(Test Accuracy: {test_accuracy*100:.2f}%)', fontsize=14)
plt.ylabel('Actual Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()

# Save
plot_folder = "history_plots"
os.makedirs(plot_folder, exist_ok=True)
cm_filename = f"cm_resnet18_{datetime.now().strftime('%Y%m%d_%H%M%S')}_Acc{test_accuracy*100:.1f}.png"
cm_filepath = os.path.join(plot_folder, cm_filename)
plt.savefig(cm_filepath, dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Confusion matrix saved to: {cm_filepath}")

In [ ]:
# =============================================================================
# CELL 16: ANALISIS CONFIDENCE
# =============================================================================

confidence_scores = np.max(Y_pred_probs, axis=1)

avg_conf = np.mean(confidence_scores)
min_conf = np.min(confidence_scores)
max_conf = np.max(confidence_scores)
std_conf = np.std(confidence_scores)

correct_mask = (y_pred == y_true)
correct_conf = np.mean(confidence_scores[correct_mask]) if correct_mask.sum() > 0 else 0
wrong_conf = np.mean(confidence_scores[~correct_mask]) if (~correct_mask).sum() > 0 else 0

print("="*60)
print("ANALISIS CONFIDENCE")
print("="*60)
print(f"\nStatistik Overall:")
print(f"  - Average Confidence: {avg_conf*100:.2f}%")
print(f"  - Min Confidence: {min_conf*100:.2f}%")
print(f"  - Max Confidence: {max_conf*100:.2f}%")
print(f"  - Std Confidence: {std_conf*100:.2f}%")
print(f"\nConfidence by Prediction:")
print(f"  - Correct predictions: {correct_conf*100:.2f}%")
print(f"  - Wrong predictions: {wrong_conf*100:.2f}%")

if correct_conf > wrong_conf + 0.05:
    print("\n✓ Model lebih confident pada prediksi yang benar (bagus!)")
else:
    print("\n⚠️ Model cukup confident bahkan pada prediksi salah")

In [ ]:
# =============================================================================
# CELL 17: SIMPAN LOG EKSPERIMEN
# =============================================================================

log_data = {
    'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Model': 'ResNet18 (Custom)',
    'Dense_Units': DENSE_UNITS,
    'Dropout': DROPOUT_RATE,
    'Fine_Tune_At': FINE_TUNE_AT,
    'Optimizer': OPTIMIZER_NAME,
    'LR_Phase_1': LR_PHASE_1,
    'Epochs_P1': EPOCHS_PHASE_1,
    'LR_Phase_2': LR_PHASE_2,
    'Epochs_P2': EPOCHS_PHASE_2,
    'Loss_Type': LOSS_TYPE,
    'Use_Class_Weights': USE_CLASS_WEIGHTS,
    'Test_Accuracy': round(test_accuracy, 4),
    'Val_Acc_P1': round(best_val_acc_p1, 4),
    'Val_Acc_P2': round(best_val_acc_p2, 4),
    'Avg_Confidence': round(avg_conf, 4),
    'Min_Confidence': round(min_conf, 4),
    'Std_Confidence': round(std_conf, 4),
    'Confusion_Matrix': cm_str,
}

# Metrik per kelas
for label in class_labels:
    log_data[f'{label}_Prec'] = round(report[label]['precision'], 4)
    log_data[f'{label}_Rec'] = round(report[label]['recall'], 4)
    log_data[f'{label}_F1'] = round(report[label]['f1-score'], 4)

# Save to CSV
df_log = pd.DataFrame([log_data])

if os.path.exists(LOG_FILE_PATH):
    df_log.to_csv(LOG_FILE_PATH, mode='a', header=False, index=False)
    print(f"✓ Log ditambahkan ke: {LOG_FILE_PATH}")
else:
    df_log.to_csv(LOG_FILE_PATH, index=False)
    print(f"✓ File log baru dibuat: {LOG_FILE_PATH}")

print("\n" + "="*60)
print("EKSPERIMEN SELESAI")
print("="*60)
print(f"\n📁 Model tersimpan di: {MODEL_SAVE_PATH}")
print(f"📁 Log tersimpan di: {LOG_FILE_PATH}")
print(f"\n🎯 Test Accuracy: {test_accuracy*100:.2f}%")

---

## 📝 Catatan Penting

### ResNet18 Custom vs Pre-trained

Model ResNet18 di notebook ini **TIDAK memiliki bobot pre-trained ImageNet**
karena kita membangunnya dari scratch.

**Solusi untuk mendapatkan pre-trained weights:**

1. **PyTorch + ONNX**: Convert dari torchvision ResNet18
2. **TensorFlow Hub**: Gunakan model dari TF Model Garden
3. **Keras Applications**: Gunakan model yang tersedia (ResNet50, EfficientNet)

### Rekomendasi

Jika hasil dari ResNet18 custom tidak memuaskan:
1. Coba **EfficientNet-B0** (5.3M params, pre-trained)
2. Coba **MobileNetV2** (3.4M params, pre-trained)
3. Coba **DenseNet121** (8M params, pre-trained)

---